In [124]:
import pandas as pd
import numpy as np
import datetime as dt
%run C:\Users\sgasmi\Notebooks\Column_autosize.ipynb

## Chargement des données

In [125]:
dhome = pd.read_excel(r"C:\Users\sgasmi\Desktop\Extractions\DATA_HOME.xlsx", sheet_name="Donnees")
fafr = pd.read_excel(r"C:\Users\sgasmi\Desktop\Extractions\Effectif_Data_Quality.xlsx", converters={'Matricule':str})

## Fonctions perso appelées

In [126]:
def empty_text(Dataframe, list_col):
    for val in list_col :
        titre_colonne = "Champ " + str(val) + " vide"
        dhome.loc[Dataframe[val].isna(), titre_colonne] = "Le champ " + str(val) + " est vide/"
    return dhome

## Contrôle de qualité du Data Home

### Traitement des champs vides

In [127]:
champs_vides = ["Email", "Sub-Domain", "Legal entity", "Establishment", "Site", "MyHR ID", "Sex", "Prof Categ", "Category", "Contract Type", "Status", "% FTE", "Job title"]
dhome = empty_text(dhome, champs_vides)

### Traitement du champ Position/coefficient

In [128]:
dhome.loc[(dhome["Position/coefficient"].isna()) & (dhome["Home Country"]=="FRA"), "Anomalie Position/coefficient vide"] = "Champ Position/coefficient vide/"

### Anomalie CSP vide

In [129]:
dhome.loc[(dhome["CSP"].isna()) & (dhome["Home Country"]=="FRA"), "Anomalie CSP"] = "Anomalie CSP vide/"

### Contrôle du champ FTE

In [130]:
dhome.loc[dhome["% FTE"]>100, "Anomalie FTE"] = "Incohérence de valeur sur le champ FTE/ "
dhome.loc[(dhome["% FTE"] !=0) & (dhome["Category"]=="B"), "Anomalie FTE"] = "Incohérence de valeur sur le champ FTE/ "
dhome.loc[(dhome["% FTE"]==0) & (dhome["Category"].isin(["A, C"])==True), "Anomalie FTE"] = "Incohérence de valeur sur le champ FTE/ "

### Contrôle de cohérence sur les expatriés

In [131]:
dhome['Anomalie expat'] = dhome.apply(lambda x : "Home Country identique à Host Country" if x['Expat/impat/detached']=="Expatriate" and x['Home Country'] == x['Host Country'] else np.nan, axis=1)

### Anomalie code UO

In [132]:
dhome.loc[dhome["ORG1 code"].isin(["ASA", "FRA", "NP"])==False, 'Incohérence du code UO niveau 1'] = "Incohérence code UO/ "

## Utilisation de la requête FAFR_035

### Typage du champ matricule

In [133]:
fafr["Matricule"] = "FRA_" + fafr["Matricule"]

### Suppression des stagiaires

In [134]:
fafr = fafr.loc[fafr["Code Contrat"] != "STA2"]
fafr.reset_index(drop=True, inplace=True)

### Détection des salariés sortis

In [135]:
dhome["local payroll ID"] = dhome["local payroll ID"].astype("str")
dhome.loc[(dhome["local payroll ID"].isin(list(fafr["Matricule"]))==False) & (dhome["Home Country"] == "FRA"), "A supprimer"] = "Salarié non présent dans FAFR35/"

### Récupération des champs dans FAFR_35

In [136]:
# Récupération du code E.T.P de la requête FAFR_35"E.T.P"
dhome = dhome.merge(fafr[["Matricule", "% rém", "E.T.P", "Code ODM", "Code section analytique", "Nombre de jours annuel"]], left_on ="local payroll ID",right_on="Matricule", how="left")

### Ajout du % FTE

In [137]:
# Si %FTE vide et FRA et % rém = 100  si % rém > 0  et non vide
dhome.loc[(dhome["% FTE"].isna()==True) & (dhome["Home Country"]=="FRA") & (dhome["% rém"]==100), "Correction code ETP"] = "Oui"
dhome.loc[(dhome["% FTE"].isna()==True) & (dhome["Home Country"]=="FRA") & (dhome["% rém"]==100), "% FTE"] = 100

# Recalcul du % rem dans FAFR35 (nombre de jours annuel (U) /212)*100
dhome.loc[(dhome["Nombre de jours annuel"].isna()==False) & (dhome["% FTE"].isna()==True), "Correction code ETP"] = "Oui"
dhome.loc[(dhome["Nombre de jours annuel"].isna()==False) & (dhome["% FTE"].isna()==True)].apply(lambda x: (x["Nombre de jours annuel"]*100)/212 if x["Home Country"]=="FRA" else "", axis=1)


# Si  %FTE vide et % rém vide et ETP non vide alors ETP * 100
dhome.loc[(dhome["% FTE"].isna()==True) & (dhome["Home Country"]=="FRA") & (dhome["% rém"].isna()==True) & (dhome["E.T.P"].isna()==False), "Correction code ETP"] = "Oui"
dhome["% FTE"] = np.where((dhome["% FTE"].isna()==True) & (dhome["Home Country"]=="FRA") & (dhome["% rém"].isna()==True) & (dhome["E.T.P"].isna()==False), dhome["E.T.P"]*100, dhome["% FTE"])

## Recherche de doublons

### Doublon matricule paie local

In [138]:
dhome.loc[dhome['local payroll ID'].duplicated()==True, "Doublon matricule paie"] = "Doublon matricule paie/ "

### Recherche de doublon MyHR

In [139]:
dhome.loc[dhome['MyHR ID'].duplicated()==True, "Doublons MyHR"] = "Doublon MyHR ID/ "

### Recherche d'homonymes

In [140]:
double = dhome.groupby(['Last name', 'First name', 'Birth date']).size().reset_index()
double.columns=['Last name', 'First name', 'Birth date', "Homonymes ou doublon"]
double= double.loc[double["Homonymes ou doublon"]>1]

dhome = dhome.merge(double, on=["Last name", "First name", "Birth date"], how='left')
dhome.loc[(dhome["Doublon matricule paie"]=="Doublon matricule paie") | (dhome["Doublons MyHR"]=="Doublon MyHR ID"), "Homonymes ou doublon"]= "X"
dhome.loc[dhome["Homonymes ou doublon"]=='X', "Homonymes ou doublon"] = ""
dhome.loc[dhome["Homonymes ou doublon"].isna()==False, "Homonymes ou doublon"] = "Homonymes ou doublon/ "

### Contrôle sub-domain écart avec PN

In [141]:
#dhome["Code ODM"] = dhome["Code ODM"].str.split("-").str[0]
#dhome["Code ODM"] = dhome["Code ODM"].str.replace(" ", "")
#dhome["Ecart sub-domain avec PN"] = dhome.apply(lambda x: "Ecart code ODM/" if x["Sub-Domain"] != x["Code ODM"] and x["Home Country"]=="FRA" and x["Legal entity"] in ["FRA", "ANP"] else np.nan, axis=1)

### Contrôle centre de coût

In [142]:
dhome["Anomalie centre de coût"] = dhome.apply(lambda x: "Ecart centre de coût/" if x["Cost Center"] != x["Code section analytique"] and x["Home Country"]=="FRA" else np.nan, axis=1)

### Contrôle du champ Category

In [143]:
dhome.loc[(dhome["Category"]=="A") & ((dhome["Contract Type"].isin(["FT3+","NFT"])==False )| (dhome["Status"].isin(["ACT","SUSP-R2"])==False) | (dhome["% FTE"]==0)), "Anomalie category"] = "Cat A incohérente/"
dhome.loc[(dhome["Category"]=="B") & ((dhome["Contract Type"] != "NFT")| (dhome["Status"].isin(["SUSP", "LONG-ILL", "SUSP-R"])==False) | (dhome["% FTE"]!=0)), "Anomalie category"] = "Cat B incohérente/"
dhome.loc[(dhome["Category"]=="C") & (dhome["Contract Type"].isin(["APP", "CP", "FT", "FTR","FTS", "FT3-","CIFRE", "FT3"])==False), "Anomalie category"] = "Cat C incohérente/"

### Champs à ajouter

In [144]:
dhome["Centres de coûts à ajouter"] = dhome.apply(lambda x: x["Code section analytique"] if x["Home Country"]=="FRA" and x["Anomalie centre de coût"]!="" else "", axis=1)
#dhome["Code ODM à ajouter"] = dhome.apply(lambda x: x["Code ODM"] if x["Home Country"]=="FRA" and x["Ecart sub-domain avec PN"]!="" else np.nan, axis=1)

### Salariés à ajouter

In [145]:
to_add = fafr.loc[fafr["Matricule"].isin(dhome["local payroll ID"])==False].reset_index(drop=True)

### Suppression des champs ajoutés par la requête FAFR35

In [146]:
dhome = dhome.drop(["Matricule", "% rém", "E.T.P", "Code ODM", "Code section analytique", "Nombre de jours annuel"], axis=1)

### Ajout du champ BU

In [147]:
dhome["BU"]=""
dhome.loc[dhome["ORG2 code"].str.startswith("OL3")==True, "BU"] = "OL3"
dhome.loc[(dhome["ORG2 code"].isin(["HR", "QP", "DGP", "ISDT", "3SEP", "RD", "OPX", "OPR"])) |(dhome["ORG2 code"].isna()==True), "BU"] = "FRA-CORP"
dhome["BU"] = dhome.apply(lambda x: x["ORG2 code"] if x["BU"]=="" else x["BU"], axis=1)

### Convertion au bon format de date

In [148]:
dhome["Contract start date"]= pd.to_datetime(dhome["Contract start date"]).dt.date
dhome["Contract end date"]= pd.to_datetime(dhome["Contract end date"]).dt.date
dhome["Birth date"]= pd.to_datetime(dhome["Birth date"]).dt.date
dhome["Group Seniority"]= pd.to_datetime(dhome["Group Seniority"]).dt.date
to_add["Date de naissance"]=pd.to_datetime(to_add["Date de naissance"]).dt.date
to_add["Début contrat initial"]=pd.to_datetime(to_add["Début contrat initial"]).dt.date
to_add["Date de fin"]=pd.to_datetime(to_add["Date de fin"]).dt.date
to_add["Ancienneté"]=pd.to_datetime(to_add["Ancienneté"]).dt.date
to_add["Ancienneté groupe"]=pd.to_datetime(to_add["Ancienneté groupe"]).dt.date

### Création d'une colonne de synthèse des anomalies

In [149]:
dhome["Synthèse anomalie"] = ""

for i, val in (dhome.iterrows()):
    synthèse = ""
    c = 71
    while c >= 45:
        if str(dhome.iloc[i,c]) != "nan":
            if synthèse == "":
                synthèse = str(dhome.iloc[i,c])
            else:
                synthèse = synthèse + str(dhome.iloc[i,c])                
        c -= 1

    if synthèse != "" :
        if synthèse !="":
            dhome.at[i, "Synthèse anomalie"]= synthèse

### Remplacement des commentaires par 1 pour les anomalies

In [150]:
#46 /71
colonne_remplace= dhome.columns[46:70]

for val in colonne_remplace:
    dhome.loc[(dhome[val].isna())==False, val]=1
    dhome.loc[dhome[val].isna()==True, val]=0
    dhome[val] = dhome[val].astype('int64')

In [151]:
log_headcount = pd.DataFrame(columns = ["MyHR ID", "Site", "Anomalie type", "Count", "Home Country", "Date"])

for col in colonne_remplace:
    prep = dhome.loc[dhome[col]==1].copy()
    prep["Anomalie type"] = col
    prep["Count"] = 1
    prep = prep[["MyHR ID", "Site", "Anomalie type", "Count", "Home Country", "Report Date"]]
    prep.columns= ["MyHR ID","Site", "Anomalie type", "Count","Home Country", "Date"]
    log_headcount = log_headcount.append(prep)
    
log_headcount = log_headcount[log_headcount["Anomalie type"] != "A supprimer"].copy()

## Export du résultat

In [152]:
with pd.ExcelWriter(r'C:\Users\sgasmi\Desktop\Export Python\Bridge_control.xlsx', date_format='dd/mm/yyyy') as writer:  
    dhome.to_excel(writer, sheet_name='Conso', index=False)
    to_add.to_excel(writer, sheet_name='Ajout', index=False)

log_headcount.to_excel(r'C:\Users\sgasmi\Desktop\Export Python\log_headcount.xlsx', index=False)

In [154]:
today = dt.date.today()
today = today.strftime('%d %m %Y')
today = today.replace(" ", "")

resize_column([log_headcount],["Anomalies"], r"C:\Users\sgasmi\Desktop\Power BI applications\Sources anomalies reporting\Anomalie_Reporting" + "_" + today +  ".xlsx")

In [ ]:
#Création d'une base de donnée in memory
conn = sqlite3.connect(':memory:')
# Ajout des Dataframe dans la base de données
grade.to_sql('grade', conn, index=False)